## Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Code

In [2]:
# https://api.mangadex.org/manga?limit=32&offset=32
# https://api.mangadex.org/manga?limit=32&offset=32+32
# https://api.mangadex.org/statistics/manga?manga[]=6f1f3a84-f2e3-4512-93bf-009fd12cfce6
# https://api.mangadex.org/manga/a15dc49f-2512-46f2-bc1e-201be8234ee5?includes[]=artist&includes[]=author&includes[]=cover_art
# https://mangadex.org/covers/a15dc49f-2512-46f2-bc1e-201be8234ee5/c20365e1-285a-4e93-b701-9fd03ff4ae19.jpg

In [3]:
# limit = 32
# offset = limit
# url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)
# response = requests.get(url)
# data = response.json()
# page_count = (round(data['total'] / 248))

# for i in range(1, page_count):
#     offset += limit
#     url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)

In [4]:
limit = 32
offset = limit
url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)
url_stat = 'https://api.mangadex.org/statistics/manga?manga[]='

def getdataurl(url):
    response = requests.get(url)
    if response.status_code == 200:
        json = []
        lan = ['en', 'ja', 'ko', 'ru', 'zh', 'ko-ro', 'zh-hk','es-la']
        data = response.json()
        
        for comic in data['data']:
            url_manga = 'https://api.mangadex.org/manga'
            url_cover = 'https://mangadex.org/covers'
            id = comic['id']
            
            title = next((v for k, v in comic['attributes']['title'].items() if k in lan), None)
            description = next((v for k, v in comic['attributes']['description'].items() if k in lan), None)
            
            alt_titles = []
            for alt_title_dict in comic['attributes'].get('altTitles', []):
                alt_title = next((v for k, v in alt_title_dict.items() if k in lan), None)
                if alt_title:
                    alt_titles.append(alt_title)
            alt_titles = ', '.join(alt_titles)
            
            genres = []
            for tag in comic['attributes'].get('tags', []):
                tag_name = next((v for k, v in tag['attributes']['name'].items() if k in ['en', 'ja', 'ko']), None)
                if tag_name:
                    genres.append(tag_name)
            genres = ', '.join(genres)
        
            released = comic['attributes']['year'] if comic['attributes']['year'] and int(comic['attributes']['year']) else '-'
            
            rate = 0
            response_rate = requests.get(url_stat+id)
            if response_rate.status_code == 200:
                data_stat = response_rate.json()
                rate = data_stat['statistics'][id]['rating']['average']

            author = ''
            artist = ''
            img = ''
            url_manga += '/'+id+'?includes[]=artist&includes[]=author&includes[]=cover_art'
            response_manga = requests.get(url_manga)
            if response_manga.status_code == 200:
                data_manga = response_manga.json()
                for relation in data_manga['data']['relationships']:
                    if relation['type'] == 'author' and 'attributes' in relation and 'name' in relation['attributes']:
                        author = relation['attributes']['name']
                    if relation['type'] == 'artist' and 'attributes' in relation and 'name' in relation['attributes']:
                        artist = relation['attributes']['name']
                    if relation['type'] == 'cover_art' and 'attributes' in relation and 'fileName' in relation['attributes']:
                        img = f"{url_cover}/{id}/{relation['attributes']['fileName']}"

            json.append({
                'title': title,
                'alt_title': alt_titles,
                'type': comic['type'],
                'description': description,
                'genre': genres,
                'author':author,
                'artist':artist,
                'rate': rate,
                'image': img,
                'released': released,
            })
            
        return json

In [5]:
df = []
limit = 32
offset = limit
url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)
data = (requests.get(url)).json()
page_count = (round(data['total'] / 248))

# get data manually 300 page's
indexNum = 0 # 901 # 701 # 601 # 501 # 401 #315 # 201 # 100 # 0
page_count = 101 # 1100 # 901 # 701 # 601 # 501 # 401 # 315 # 201 # 100

for i in range(indexNum, page_count):
    offset += limit
    url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)
    getdata = getdataurl(url)
    df = df + getdata

df = pd.DataFrame(df)
df

,title,alt_title,type,description,genre,author,artist,rate,image,released
0,The Hero Returns,"Hero, Returns, 영웅, 회귀하다, Герой возвращается, H...",manga,"The strongest hero of humanity, Kim Sung Bin.\...","Time Travel, Monsters, Action, Long Strip, Adv...",B. Ain (흑아인),Yoon Goon (윤군),8.1263,https://mangadex.org/covers/e0966537-58a1-421b...,2022
1,Juujika no Rokunin,"La cruz de seis personas, Six People on Crosse...",manga,"”Is it okay to kill people?"" \nA treacherous ...","Thriller, Psychological, Sexual Violence, Hare...",Nakatake Shiryuu,Nakatake Shiryuu,7.7258,https://mangadex.org/covers/00446151-d199-45d6...,2020
2,Gensou Gourmet,"Fantasy Gourmet, Gourmet in different world., ...",manga,"One day, Shunichirou suddenly transferred to a...","Monsters, Demons, Comedy, Office Workers, Isek...",Amana Kouta,Otsuji,8.4639,https://mangadex.org/covers/ede48b14-8d83-41cb...,2016
3,Release That Witch,"Освободите эту Ведьму, 放开那个女巫",manga,A male engineer transmigrated into another wor...,"Action, Demons, Long Strip, Romance, Adventure...",Er Mu,Dr. Woodman,8.1814,https://mangadex.org/covers/aafb046c-9de4-4c3d...,2018
4,Jack Jeanne: PUPPET,"PUPPET, Марионетка",manga,None,"Oneshot, Drama, Mystery",Ishida Sui,Ishida Sui,6.6000,https://mangadex.org/covers/9065edbb-87ce-4286...,2022
...,...,...,...,...,...,...,...,...,...,...
3227,The Hundred Ghost Stories That Led to My Death,"僕が死ぬだけの百物語, Boku ga Shinu dake no Hyakumonogat...",manga,"One day, an elementary school child named Yuum...","Monsters, Demons, Psychological, Ghosts, Antho...",Matono Anji,Matono Anji,8.8543,https://mangadex.org/covers/1c01dcad-8e60-4fdd...,2020
3228,Kimi to Koi no Tochuu,"Partway Through Love With You, キミと恋の途中",manga,"Just when I thought it'd start with you, you d...","Romance, Comedy, Drama, School Life, Slice of ...",Aikawa Saki,Aikawa Saki,5.8000,https://mangadex.org/covers/d30e7385-277f-40fc...,2010
3229,Division Chief Shima Kōsaku,"Shima Kōsaku Part 2, 部長島耕作",manga,Division Chief Shima Kōsaku is the second part...,"Psychological, Romance, Crime, Mafia, Office W...",Hirokane Kenshi,Hirokane Kenshi,9.1053,https://mangadex.org/covers/61334d85-fb98-465a...,1992
3230,Zom 100 ~100 Things I Want to do Before I Beco...,"Zom 100: Bucket List of the Dead, Zombie 100 ~...",manga,After three years of working for a black compa...,"Action, Comedy, Zombies, Adventure, Gore, Dram...",Asou Haro,Takata Koutarou,8.7425,https://mangadex.org/covers/efd02206-1cd8-4823...,2018


In [6]:
df.to_csv('data/mangadex.csv', index=False, encoding='utf-8')